## New York City Taxi Fare Prediction
### Can you predict a rider's taxi fare?
<br>
<br>
This is one of the Kaggle Competition for biginner, alredy completed.<br>
Even it is very simple and straitforward for estimation, i thought it will be a good example to practice estimation problem from the scrath. <br>

You can refer to here for details, https://www.kaggle.com/c/new-york-city-taxi-fare-prediction <br>
This is a tensorflow code I made for your reference,
Your task is to create PyTorch code and run your own code, and compare it with other competitions.

If you have any questions on this implementaion, email [me](jychoi.ethan@gmail.com)

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

#!pip install -q tensorflow==2.0.0-beta1
import sys, os
import pandas as pd
import tensorflow as tf
import numpy as np

## Read data using pandas
set path of the csv file containing the heart dataset.

In [2]:
files = os.listdir('./')
files

['train_s.csv',
 '.DS_Store',
 'NewYork_City_Taxi_Fare_Prediction_tf.ipynb',
 'test.csv',
 'submission.csv',
 'GCP-Coupons-Instructions.rtf',
 'train.csv',
 '.ipynb_checkpoints',
 'make_train_s.ipynb',
 'sample_submission.csv',
 'NewYork_City_Taxi_Fare_Prediction.ipynb']

### The train file is too large and only 10% of the samples are imported for this training.
```
# the original size of train data is 55423856, about 55M
df = df_train.sample(frac=0.1, replace=True, random_state=1)
print('The size of train data =', len(df))

df.to_csv('./train_s.csv')
```

### Please download pre-created train_s file which samples 10% of the original file randomly
**train_s.csv file download** 
[train_s.csv](https://drive.google.com/uc?export=download&id=1AbO1jfrwJ0IKSJQactC6msPNub131LAh)
<br>**test.csv file download**
[test_s.csv](https://drive.google.com/uc?export=download&id=1iE_JilybsBIBpaTveD6vX8AfwaZzI8i8)

Read the csv file using pandas.<br>
Instead of using original train.csv file, use smaller version, train_s.csv

In [3]:
#train_csv_file = './train.csv'
train_csv_file = './train_s.csv'
test_csv_file = './test.csv'

### Let's read csv data into pandas dataframe
It will take some time to load as train.csv is a bit huge, please be patient 

In [4]:
df_train = pd.read_csv(train_csv_file, index_col=0)
df_test = pd.read_csv(test_csv_file, index_col=0)
print('The size of train data =', len(df_train))
print('The size of test =', len(df_test))
print('The types of data =', df_train.dtypes)

The size of train data = 5542386
The size of test = 9914
The types of data = fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object


**Let's see what the data looks like**

In [5]:
df_train.head(2)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,,
2013-03-02 18:04:10.0000006,18.0,2013-03-02 18:04:10 UTC,-73.988898,40.721670,-73.985762,40.768975,1
2009-06-12 23:00:29.0000006,9.7,2009-06-12 23:00:29 UTC,-73.985116,40.742225,-73.998634,40.763971,3


### Data preprocess
- Drop columns not to be used for features, rows which has NaN values
- Remove outliers
- apply MinMaxScaler if possible

In [6]:
# Let's look over train data if there are Nan in any of features
print(df_train.isnull().sum() , df_test.isnull().sum())

fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    33
dropoff_latitude     33
passenger_count       0
dtype: int64 pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64


In [7]:
# Remove missing values
df_train = df_train.dropna(how = 'any')

### From [the starter code](https://www.kaggle.com/dster/nyc-taxi-fare-starter-kernel-simple-linear-model) 

Add columns

```
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train_df)
```

In [8]:
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

# add longitude, latitude difference to column features
add_travel_vector_features(df_train)
add_travel_vector_features(df_test)

### Filtering out outliers
From the starter code<br>
"We expect most of these values to be very small (likely between 0 and 1) since it should all be differences between GPS coordinates within one city. For reference, one degree of latitude is about 69 miles. However, we can see the dataset has extreme values which do not make sense. Let's remove those values from our training set. Based on the scatterplot, it looks like we can safely exclude values above 5"

In [9]:
df_train = df_train[(df_train.abs_diff_longitude < 5.0) & (df_train.abs_diff_latitude < 5.0)]

In [10]:
## Let's see the size of data again now
print('The size of train data =', len(df_train))
print('The size of test =', len(df_test))
print('The types of data =', df_train.dtypes)

The size of train data = 5530809
The size of test = 9914
The types of data = fare_amount           float64
pickup_datetime        object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
passenger_count         int64
abs_diff_longitude    float64
abs_diff_latitude     float64
dtype: object


### Set columns to drop

In [11]:
def preprocess_data(df_train, df_test, cols_to_drop='', label='fare_amount'):
    """
    Returns df_train_x, df_train_y, df_test_x
    """
    drop_cols = []
    for col in cols_to_drop:
        if col in df_train:
            drop_cols.append(col)

    train_x = df_train.drop(columns=drop_cols)
    train_y = train_x.pop(label)
    
    test_x = df_test.drop(columns=drop_cols)

    return (train_x, train_y), test_x

In [12]:
# columns to drop in input features
# simply not to use pickup_datetime as a feature for estimation here.
COLUMNS_TO_DROP = ['pickup_datetime', 'pickup_longitude', 'pickup_latitude', \
                   'dropoff_longitude', 'dropoff_latitude']

In [13]:
df_train.columns.values

array(['fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'abs_diff_longitude', 'abs_diff_latitude'],
      dtype=object)

In [14]:
(train_x, train_y), test_x = \
        preprocess_data(df_train, df_test, COLUMNS_TO_DROP, 'fare_amount')

In [15]:
print(train_x.shape, train_y.shape, test_x.shape)

(5530809, 3) (5530809,) (9914, 3)


In [16]:
train_x.head(2)

,passenger_count,abs_diff_longitude,abs_diff_latitude
key,,,
2013-03-02 18:04:10.0000006,1,0.003136,0.047305
2009-06-12 23:00:29.0000006,3,0.013518,0.021746


### Let's find out min, max values of each columns in train, test set

In [17]:
train_x_min = train_x.min(axis=0)
train_x_max = train_x.max(axis=0)
test_x_min = test_x.min(axis=0)
test_x_max = test_x.max(axis=0)

In [18]:
x_min = train_x_min.combine(test_x_min, min)
x_max = train_x_max.combine(test_x_max, max)

In [19]:
x_max

passenger_count       208.000000
abs_diff_longitude      4.989833
abs_diff_latitude       4.941050
dtype: float64

In [20]:
x_min

passenger_count       0.0
abs_diff_longitude    0.0
abs_diff_latitude     0.0
dtype: float64

In [21]:
def _MinMaxScaler(X, s_min, s_max, feature_range=(0, 1)):
    
    min, max = feature_range
    
    episilon = 1e-10
    X_std = (X - s_min) / (s_max - s_min + episilon)
    X_scaled = X_std * (max - min) + min
    
    return X_scaled

### Apply MinMaxScaler to train and test dataframe
(This is optional)

In [22]:
#train_x = _MinMaxScaler(train_x, x_min, x_max)
#test_x = _MinMaxScaler(test_x, x_min, x_max)

In [23]:
train_x.head(2)

,passenger_count,abs_diff_longitude,abs_diff_latitude
key,,,
2013-03-02 18:04:10.0000006,1,0.003136,0.047305
2009-06-12 23:00:29.0000006,3,0.013518,0.021746


## Load data using `tf.data.Dataset`

Use tf.data.Dataset.from_tensor_slices to read the values from a pandas dataframe.

One of the advantages of using tf.data.Dataset is it allows you to write simple, highly efficient data pipelines. Read the loading data guide to find out more.

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((train_x.values, train_y.values))

For reading<br>
```for feat, targ in dataset.take(2):
       print ('Features: {}, Target: {}'.format(feat, targ))```

Shuffle and batch the dataset.

In [25]:
BATCH_SIZE = 128

In [26]:
train_dataset = dataset.shuffle(len(train_x)).batch(BATCH_SIZE)
print(train_dataset)
_, input_size = train_x.shape

<BatchDataset shapes: ((None, 3), (None,)), types: (tf.float64, tf.float64)>


## Create and train a model

In [27]:
def get_compiled_model(hidden_units, activation='relu', lr=1e-3, l2_scale=1e-3):
    model = tf.keras.Sequential()
    
    for units in hidden_units:
        model.add(tf.keras.layers.Dense(units, \
                activation=activation, kernel_regularizer=tf.keras.regularizers.l2(l2_scale)))
    
    model.add(tf.keras.layers.Dense(1, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers.l2(l2_scale)))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

In [28]:
model = get_compiled_model([input_size, 16, 8], lr=1e-3, l2_scale=1e-4)

## Training

In [29]:
# Just run 2 times as a trial
model.fit(train_dataset, epochs=2)

Epoch 1/2
43210/43210 [==============================] - 100s 2ms/step - loss: 47.6746 - mae: 3.4343
Epoch 2/2
43210/43210 [==============================] - 95s 2ms/step - loss: 43.5171 - mae: 2.7183


In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  12        
_________________________________________________________________
dense_1 (Dense)              multiple                  64        
_________________________________________________________________
dense_2 (Dense)              multiple                  136       
_________________________________________________________________
dense_3 (Dense)              multiple                  9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [31]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_x.values))
test_dataset = test_dataset.batch(BATCH_SIZE)

## Model predict for test set

In [32]:
fare_predict = model.predict(test_dataset)

In [33]:
print('the size of fare_predict=', len(fare_predict))
print('the shape=', fare_predict.shape)

the size of fare_predict= 9914
the shape= (9914, 1)


In [34]:
fare_predict = fare_predict.reshape(-1)
key = df_test.index.values
print(key.shape, fare_predict.shape)

(9914,) (9914,)


In [35]:
# Write the predictions to a CSV file which we can submit to the competition.
submission = pd.DataFrame(
    {'key': df_test.index, 'fare_amount': fare_predict},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)